# Intro
Neural networks comprise of layers of neurons and connections between the neurons of each layer. Tuning the weights and biases of these connections allow the network to "learn" and predict.

# Neuron code
Suppose we are looking at a single neuron taking in 3 inputs from the previous layer. The following is a simplified look at what a neuron does. It takes a weighted sum of its inputs and adds the bias associated with the neuron. The result of this calculation is output of the neuron.

In [3]:
inputs = [1, 2, 3] # Output of previous layer's neurons (could be from an actual input layer or a hidden layer)
weights = [0.2, 0.8, -0.5] # Strength of connection between the previous layer's neurons
bias = 2 # Bias associated with this particular neuron

output = inputs[0]*weights[0] + inputs[1]*weights[1] + inputs[2]*weights[2] + bias
print(output)

2.3


# Layer Code
Now suppose we are are looking at a single layer consisting of 3 neurons taking inputs from a previous layer of 4 neurons. This time there would be 3 sets of weights as well as 3 biases.

In [4]:
inputs = [1, 2, 3, 2.5]
weights1 = [0.2, 0.8, -0.5, 1.0]
weights2 = [0.5, -0.91, 0.26, -0.5]
weights3 = [-0.26, -0.27, 0.17, 0.87]

bias1 = 2
bias2 = 3
bias3 = 0.5

output = [inputs[0]*weights1[0] + inputs[1]*weights1[1] + inputs[2]*weights1[2] + inputs[3]*weights1[3] + bias1,
		  inputs[0]*weights2[0] + inputs[1]*weights2[1] + inputs[2]*weights2[2] + inputs[3]*weights2[3] + bias2,
		  inputs[0]*weights3[0] + inputs[1]*weights3[1] + inputs[2]*weights3[2] + inputs[3]*weights3[3] + bias3]
print(output)

[4.8, 1.21, 2.385]
